## Review *original* paradigm - generated by Optseq

In [ ]:
import os
import csv
import pandas as pd

par_path = '/Users/emilyhahn/projects/newARC/optseq/'
# par_files=['3cond-001.par','3cond-002.par','3cond-003.par']
par='3cond-001.par'
par_out='3cond-001.csv'

par1={'Duration':[],'Condition':[],'ITI':[],'Slope':[],'ConflictType':[]}
# par2={'time':[],'cond':[],'ITI':[],'slope':[],'type':[]}
# par3={'time':[],'cond':[],'ITI':[],'slope':[],'type':[]}

ITI_NULL=[]
ITI_noNULL=[]

with open(os.path.join(par_path,par),'r') as par:
    for line in par:
#         line=str(line).split('    ')
#         whole_line=line[0:47]
        duration=line[0:8]
        cond=line[12:14]
        ITI=line[16:22]
        slope=line[25:28]
        type=line[33:47]
        par1['Duration'].append(duration.strip())
        par1['Condition'].append(cond.strip())
        par1['ITI'].append(ITI.strip())
        par1['Slope'].append(slope.strip())
        par1['ConflictType'].append(type.strip())
        if 'NULL' in line:
            ITI_NULL.append(float(ITI.strip()))
        elif 'NULL' not in line:
            ITI_noNULL.append(float(ITI.strip()))
        
# print sum(ITI_noNULL)
# print sum(ITI_NULL)
# print sum(ITI_noNULL+ITI_NULL)

df=pd.DataFrame.from_dict(par1)
df.to_csv(os.path.join(par_path,par_out))

## Review *existing* paradigm - input to PsychToolbox

In [ ]:
import os
import pandas as pd

df = pd.read_csv('mri_newarc.csv',index_col=False)
df=pd.DataFrame(df)
display(df)

trial_1 = df.iloc[0:81]
trial_2 = df.iloc[81:162]
trial_3 = df.iloc[162::]

# print trial_1['ITI']

block1_time_sec = sum((trial_1['ITI'])+(5.25*81))
block1_time_min = (sum(trial_1['ITI'])+(5.25*81))/60

block2_time_sec = sum((trial_2['ITI'])+(5.25*81))
block2_time_min = (sum(trial_2['ITI'])+(5.25*81))/60

block3_time_sec = sum((trial_3['ITI'])+(5.25*81))
block3_time_min = (sum(trial_3['ITI'])+(5.25*81))/60


print 'Block 1 | ITI sum: %s' % sum(trial_1['ITI'])
print '        | Total Block Time: %s sec (%s minutes)' % (block1_time_sec,block1_time_min)
print '        | Risk sum: %s' % sum(trial_1['Risk'])
print '        | reward sum: %s \n' % sum(trial_1['Reward'])

print 'Block 2 | ITI sum: %s' % sum(trial_2['ITI'])
print '        | Total Block Time: %s sec (%s minutes)' % (block2_time_sec,block2_time_min)
print '        | Risk sum: %s' % sum(trial_2['Risk'])
print '        | Reward sum: %s \n' % sum(trial_2['Reward'])

print 'Block 3 | ITI sum: %s' % sum(trial_3['ITI'])
print '        | Total Block Time: %s sec (%s minutes)' % (block3_time_sec,block3_time_min)
print '        | Risk sum: %s' % sum(trial_3['Risk'])
print '        | Reward sum: %s' % sum(trial_3['Reward'])

# print trial_1['ITI'].value_counts()

## Generate subject list - input to **mat2csv.m**

In [ ]:
raw_dir='/Users/emilyhahn/projects/newARC/behavior/params_raw/'

sub_mat_list=[]

for root, dirs, files in os.walk(raw_dir, topdown=False):
    for name in files:
        sub_mat_list.append(name) 
        
# print sub_mat_list

## Configure variables to determine missing data

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib as plt
# import tables

standard_vars=['sub_csv_list','subjs_missing_cols','subjs_missing_cols_csv']

port=['missing','partial','all','other']
block=[1,2,3]
modality=['eeg','mri']
d={}

for mod in modality:
    d["{0}_subjs".format(mod)]=[]

for x in standard_vars:
    d["{0}".format(x)]=[]

for x in port:
    for y in block:
        for mod in modality:
            d["{0}_trial_{1}_{2}".format(mod,y,x)]=[]
            d["{0}_trial_{1}_{2}".format(mod,y,x)]=[]
            
dir='/Users/emilyhahn/projects/newARC/behavior/preprocessed/'

col_list=['ConflictType','ITI','Reward','Risk','Trial','TimeType',\
          'RiskType','RewardType','Shock','LeftKey','FixOnset',\
          'RiskOnset','RewardOnset','ResponseType','ResponseOnset',\
          'ResponseTime', 'ResponseUncertainty','ResultOnset','ShockOnset']
                
for root, dirs, files in os.walk(dir, topdown=False):
    for name in files:
         d['sub_csv_list'].append(name)

## Determine missing data

In [ ]:
# print 'length eeg and mri ORIGINAL subject list: %s' % len(d['sub_csv_list'])
        
for csv in d['sub_csv_list']:
    f=os.path.join(dir,csv)
    df_x=pd.read_csv(f,index_col=None)
    df=pd.DataFrame(df_x)
    for col in col_list:
        ## Detect missing columns
        if col not in df.columns and 'eeg' not in csv:
            d['subjs_missing_cols'].append('%s' % csv)
            d['subjs_missing_cols_csv'].append('%s missing column: %s\n' % (csv,col))
            print '%s missing column: %s\n' % (csv,col)

# Refine sub_csv_list
# for i in d['subjs_missing_cols']:
#     if i in d['sub_csv_list']:
#         d['sub_csv_list'].remove(i)
        
# print 'length eeg and mri TRIMMED subject list: %s\n\n' % len(d['sub_csv_list'])

for csv in d['sub_csv_list']:
    if 'hc017' not in csv:
        z=os.path.join(dir,csv)
        df_x=pd.read_csv(z,index_col=None)
        df=pd.DataFrame(df_x)
        trial_1 = df.iloc[0:81]
        trial_2 = df.iloc[81:162]
        trial_3 = df.iloc[162::]
        if 'mri' in csv:
            d['mri_subjs'].append(csv)
            print "\n ------ %s ------" % csv
            if 'block_1' in csv or "block" not in csv:
                if len(trial_1['ResponseTime']) != 81:
                    print "length of block 1: %s " % len(trial_1['ResponseTime'])
                if trial_1['ResponseTime'].isnull().all()==True:
                    d['mri_trial_1_missing'].append(csv)
                    print "ALL trials in block 1 are null - REVIEW"
                elif trial_1['ResponseTime'].isnull().any()==True:
                    d['mri_trial_1_partial'].append(csv)
                    print "SOME trials in block 1 are null - REVIEW"
                elif trial_1['ResponseTime'].isnull().all()==False:
                    d['mri_trial_1_all'].append(csv)
    #                 print "NO trials in block 1 are null - GOOD"
                else:
                    mri_trial_1_other.append(csv)
            elif 'block_2' in csv:
                if len(trial_2['ResponseTime']) != 81:
                    print "length of block 2: %s " % len(trial_2['ResponseTime'])
                if trial_2['ResponseTime'].isnull().all()==True:
                    d['mri_trial_2_missing'].append(csv)
                    print "ALL trials in block 2 are null - REVIEW"
                elif trial_2['ResponseTime'].isnull().any()==True:
                    d['mri_trial_2_partial'].append(csv)
                    print "SOME trials in block 2 are null - REVIEW"
                elif trial_2['ResponseTime'].isnull().all()==False:
                    d['mri_trial_2_all'].append(csv)
    #                 print "NO trials in block 2 are null - GOOD"
                else:
                    mri_trial_2_other.append(csv)
            elif 'block_3' in csv:
                if len(trial_3['ResponseTime']) != 81:
                    print "length of block 3: %s " % len(trial_3['ResponseTime'])
                if trial_3['ResponseTime'].isnull().all()==True:
                    d['mri_trial_3_missing'].append(csv)
                    print "ALL trials in block 3 are null - REVIEW"
                elif trial_3['ResponseTime'].isnull().any()==True:
                    d['mri_trial_3_partial'].append(csv)
                    print "SOME trials in block 3 are null - REVIEW"
                elif trial_3['ResponseTime'].isnull().all()==False:
                    d['mri_trial_3_all'].append(csv)
    #                 print "NO trials in block 3 are null - GOOD"
                else:
                    mri_trial_3_other.append(csv)
            else:
                print "Check subject %s" % csv

        elif 'eeg' in csv:
            d['eeg_subjs'].append(csv)
    #         if trial_1['ResponseTime'].isnull().all()==True:
    #             d['eeg_trial_1_missing'].append(csv)
    #         elif trial_1['ResponseTime'].isnull().any()==True:
    #             d['eeg_trial_1_partial'].append(csv)
    #         elif trial_1['ResponseTime'].isnull().all()==False:
    #             d['eeg_trial_1_all'].append(csv)
    #         else:
    #             d['eeg_trial_1_other'].append(csv)
    elif 'hc017' in csv:
        print "\nskipping hc017, %s\n" % csv
        continue

## Summarize complete, partial, and missing data

In [ ]:
# print d['subjs_missing_cols_csv']
# print d['subjs_missing_cols']

# print 'length of mri_subjs: %s ' % len(d['mri_subjs'])
# print 'length mri_trial_1_other: %s' % len(d['mri_trial_1_other'])   
# print 'length mri_trial_1_missing: %s' % len(d['mri_trial_1_missing'])
# print 'length mri_trial_1_partial: %s' % len(d['mri_trial_1_partial'])
# print 'length mri_trial_1_all: %s' % len(d['mri_trial_1_all'])
# print 'sum of mri_trial_1_missing, mri_trial_1_partial, mri_trial_1_all: %s \n\n' % (len(d['mri_trial_1_missing']) + \
#                                             len(d['mri_trial_1_partial']) \
#                                             + len(d['mri_trial_1_all']))

# print 'length of eeg_subjs: %s \n\n' % len(d['eeg_subjs']) 
# print 'length eeg_trial_1_other: %s' % len(d['eeg_trial_1_other']) 
# print 'length eeg_trial_1_missing: %s' % len(d['eeg_trial_1_missing'])
# print 'length eeg_trial_1_partial: %s' % len(d['eeg_trial_1_partial'])
# print 'length eeg_trial_1_all: %s' % len(d['eeg_trial_1_all'])
# print 'sum of eeg_trial_1_missing, eeg_trial_1_partial, eeg_trial_1_all: %s \n\n' \
#                                             % (len(d['eeg_trial_1_missing']) + \
#                                             len(d['eeg_trial_1_partial']) \
#                                             + len(d['eeg_trial_1_all']))

## Manually review erroneously recorded behavior files

In [ ]:
# null=['hc005_ARC_mri_2018_04_22_10_33_36.csv',\
#      'hc005_ARC_mri_2018_04_22_10_56_13.csv',\
#      'hc009_ARC_mri_block_1_2018_05_05_10_43_23.csv',\
#      'hc009_ARC_mri_block_2_2018_05_05_10_58_33.csv',\
#      'hc009_ARC_mri_block_3_2018_05_05_11_32_51.csv',\
#      'hc019_ARC_mri_block_1_2019_01_04_10_59_45.csv']
      
for review in null:
    z=os.path.join(dir,review)
    df_x=pd.read_csv(z,index_col=None)
    df=pd.DataFrame(df_x)
#     trial_1 = df.iloc[0:81]
#     trial_2 = df.iloc[81:162]
#     trial_3 = df.iloc[162::]
    print review
    pd.options.display.max_rows
    pd.set_option('display.max_rows',500)
    display(df)

## Combine subjects to one xlsx for report purposes

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib as plt
import csv
import xlwt
import openpyxl
from openpyxl import Workbook
import glob, os
import pandas as pd
from pandas import DataFrame, ExcelWriter

import pandas as pd
import numpy as np
import os
import matplotlib as plt
# import tables

standard_vars=['sub_csv_list','subjs_missing_cols','subjs_missing_cols_csv']

port=['missing','partial','all','other']
block=[1,2,3]
modality=['eeg','mri']
d={}

for mod in modality:
    d["{0}_subjs".format(mod)]=[]
    
for x in standard_vars:
    d["{0}".format(x)]=[]

for x in port:
    for y in block:
        for mod in modality:
            d["{0}_trial_{1}_{2}".format(mod,y,x)]=[]
            d["{0}_trial_{1}_{2}".format(mod,y,x)]=[]
            
dir='/Users/emilyhahn/projects/newARC/behavior/preprocessed/'
                
for root, dirs, files in os.walk(dir, topdown=False):
    for name in files:
         d['sub_csv_list'].append(name)

writer = ExcelWriter(os.path.join(dir,"combined_subejcts.xlsx"))

mri_list=[]

for csv in d['sub_csv_list']:
    if 'mri' in csv:
        mri_list.append(csv)

mri_list=sorted(mri_list)        

for csv in mri_list:
    new_id=csv[:-4].replace('_ARC_mri','').replace('_2018','').replace('block','run')
    df_csv = pd.read_csv(os.path.join(dir,csv))
#     print csv
#     print new_id
#     display(df_csv.head())
    (_, f_name) = os.path.split(new_id)
    (f_short_name, _) = os.path.splitext(new_id)
    df_csv.to_excel(writer, f_short_name, index=False)
    print '%s ADDED to combined_subejcts.xlsx\n\n' % new_id 
writer.save()